# Load libraries and source files

In [9]:
require(tidyverse)
source("src/simulation.libs.R")

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



Set up function for dispatching jobs

In [10]:
runSimChunk <- function(save, param, rep, email = F) {
    # Create output filename
    outName <- paste0("sim", param["cells"], "cells_", param["donors"], "donors_", param["foldchange"], "foldchange_", "rep", rep, ".rds")
    if(email == T){
        apply(param, 1, function(x) {
            str <- paste0('bsub -q short Rscript src/simulateCells.R', 
            ' --ndonors ', x['donors'], 
            ' --ncells ', x['cells'], 
            ' --foldchange ', x['foldchange'],
            ' --filename ', outName,
            ' -o ', save)
            system(str)
        })
    } else {
        apply(param, 1, function(x) {
            str <- paste0('LSB_JOB_REPORT_MAIL=N bsub -q short Rscript src/simulateCells.R', 
            ' --ndonors ', x['donors'], 
            ' --ncells ', x['cells'], 
            ' --foldchange ', x['foldchange'], 
            ' --filename ', outName,
            ' -o ', save)
            system(str)
        })
    }   
}

# Set up test parameter seach space

In [11]:
donors <- c(50, 100)
cells <- 1000
foldchange <- c(1.5, 2)

params <- expand.grid(donors, cells, foldchange)
names(params) <- c("donors", "cells", "foldchange")

# Simulate data from parameter settings, using replicates

In [33]:
for (i in 1:nrow(params)) {
    setting <- params[i,]
    for (k in 1:replicates) {
        runSimChunk(save = "test", param = setting, rep = k, email = F)
    }
    while (length(system("bjobs", intern=T)) != 0) {
        sleep(60)
    }
}

ERROR: Error in sleep(60): could not find function "sleep"


In [32]:
params

donors,cells,foldchange
50,1000,1.5
100,1000,1.5
50,1000,2.0
100,1000,2.0
